In [3]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 设置随机种子以确保可重复性
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

# 公司名称列表
company_name_list = [
    "湖北国贸能源化工有限公司",
    "湖北国贸金属矿产有限公司",
    "湖北国贸汽车有限公司",
    "湖北国际贸易集团有限公司",
    "湖北国贸农产品有限公司",
    "武汉鼎联丰国际贸易有限公司",
    "湖北国贸农产品有限公司武汉分公司",
    "湖北南方大集实业有限公司",
    "湖北南方大集实业有限公司东西湖分公司",
    "湖北南方大集实业有限公司慈惠分公司",
    "湖北南方大集实业有限公司江汉分公司",
    "湖北南方大集实业有限公司能源分公司",
    "湖北南方工贸有限公司",
    "湖北南方集团有限公司",
    "湖北国贸供应链管理有限公司",
    "湖北华中能源发展有限公司",
    "湖北国贸汽车有限公司红安分公司",
]

# 生成随机客户名称
customer_names = [f"客户{i}" for i in range(1, 51)]

# 生成数据
n_rows = 1000
now = datetime.now()
data = {
    "公司名称": np.random.choice(company_name_list, n_rows),
    "客户名称": np.random.choice(customer_names, n_rows),
    "销售金额": np.random.uniform(100000, 5000000, n_rows).round(2),
    "销售结算日期": [
        (now - timedelta(days=random.randint(0, 365))).strftime("%Y-%m-%d")
        for _ in range(n_rows)
    ],
    "商品类别": np.random.choice(["煤炭", "钢材"], n_rows),
}

# 创建 DataFrame
df = pd.DataFrame(data)

# 添加采购金额和采购结算日期
df['采购金额'] = df['销售金额'] * np.random.uniform(0.7, 0.9, n_rows)
df['采购结算日期'] = df['销售结算日期'].apply(lambda x: (datetime.strptime(x, "%Y-%m-%d") - timedelta(days=random.randint(30, 90))).strftime("%Y-%m-%d"))

# 计算毛利润和毛利率
df['毛利润'] = df['销售金额'] - df['采购金额']
df['毛利率'] = df['毛利润'] / df['销售金额']

# 将金额四舍五入到两位小数
df['销售金额'] = df['销售金额'].round(2)
df['采购金额'] = df['采购金额'].round(2)
df['毛利润'] = df['毛利润'].round(2)
df['毛利率'] = df['毛利率'].round(4)  # 毛利率保留4位小数
print(df.head)

<bound method NDFrame.head of                   公司名称  客户名称        销售金额      销售结算日期 商品类别        采购金额  \
0     湖北国贸农产品有限公司武汉分公司  客户24  4537071.48  2023-10-14   钢材  3370742.72   
1        湖北国贸供应链管理有限公司  客户22  3885851.36  2024-07-10   钢材  3236909.92   
2    湖北南方大集实业有限公司江汉分公司   客户5  1397987.69  2024-08-24   钢材  1117842.25   
3         湖北南方大集实业有限公司  客户33  3261920.20  2024-04-18   煤炭  2767951.30   
4     湖北国贸农产品有限公司武汉分公司  客户36  3019178.57  2024-05-03   煤炭  2310906.24   
..                 ...   ...         ...         ...  ...         ...   
995   湖北国贸农产品有限公司武汉分公司  客户19  1585650.89  2024-04-23   煤炭  1426105.62   
996       湖北南方大集实业有限公司   客户4  1520853.58  2023-10-21   钢材  1120398.35   
997         湖北国贸汽车有限公司  客户26  2860115.76  2023-11-09   钢材  2341943.93   
998       湖北国贸金属矿产有限公司  客户38   687773.10  2023-10-03   钢材   538027.64   
999      湖北国贸供应链管理有限公司  客户42  4592279.28  2024-05-27   钢材  3330675.85   

         采购结算日期         毛利润     毛利率  
0    2023-08-18  1166328.76  0.2571  
1    2024-06-03  

In [4]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# 假设df是我们之前生成的DataFrame
df['销售结算日期'] = pd.to_datetime(df['销售结算日期'])

def set_mobile_layout(fig, title, x_title, y_title):
    fig.update_layout(
        autosize=False,
        width=360,  # 假设手机屏幕宽度为360px
        height=300,  # 大约半个手机屏幕的高度
        margin=dict(l=10, r=10, t=60, b=40),
        title_font_size=16,
        font=dict(size=12),
        title=dict(text=title, x=0.3),
        xaxis=dict(
            title=x_title,
            tickangle=-45,
            title_font=dict(size=14),
            tickfont=dict(size=10)
        ),
        yaxis=dict(
            title=y_title,
            title_font=dict(size=14),
            tickfont=dict(size=10),
            tickformat='.2s'  # 使用科学记数法显示大数字
        )
    )
    return fig

def calculate_bar_width(n_bars, chart_width=360, min_width=0.5, max_width=0.8):
    # 计算理想宽度
    ideal_width = (chart_width * 0.8) / (n_bars + 1)  # 0.8是为了留一些边距
    # 将宽度限制在最小和最大值之间
    return max(min(ideal_width / chart_width, max_width), min_width)

# 创建公司名称简称的字典
company_short_names = {
    "湖北国贸能源化工有限公司": "国贸能源",
    "湖北国贸金属矿产有限公司": "国贸金属",
    "湖北国贸汽车有限公司": "国贸汽车",
    "湖北国际贸易集团有限公司": "国贸集团",
    "湖北国贸农产品有限公司": "国贸农产",
    "武汉鼎联丰国际贸易有限公司": "鼎联丰",
    "湖北国贸农产品有限公司武汉分公司": "国贸农产武汉",
    "湖北南方大集实业有限公司": "南方大集",
    "湖北南方大集实业有限公司东西湖分公司": "南方大集东西湖",
    "湖北南方大集实业有限公司慈惠分公司": "南方大集慈惠",
    "湖北南方大集实业有限公司江汉分公司": "南方大集江汉",
    "湖北南方大集实业有限公司能源分公司": "南方大集能源",
    "湖北南方工贸有限公司": "南方工贸",
    "湖北南方集团有限公司": "南方集团",
    "湖北国贸供应链管理有限公司": "国贸供应链",
    "湖北华中能源发展有限公司": "华中能源",
    "湖北国贸汽车有限公司红安分公司": "国贸汽车红安",
}

# 将简称添加到DataFrame中
df['公司简称'] = df['公司名称'].map(company_short_names)


In [5]:


# 例子1: 集团用户查询今年上半年的销售情况
current_year = pd.Timestamp.now().year
df_half_year = df[(df['销售结算日期'].dt.year == current_year) & (df['销售结算日期'].dt.month <= 6)]
df_company_sales = df_half_year.groupby('公司名称')['销售金额'].sum().sort_values(ascending=False).reset_index()
df_company_sales['公司简称'] = df_company_sales['公司名称'].map(company_short_names)

fig1 = px.bar(df_company_sales, x='公司简称', y='销售金额')
fig1 = set_mobile_layout(fig1, 
                         f'{current_year}年上半年<br>各公司销售情况', 
                         '公司名称', '销售额 (元)')
fig1.update_xaxes(type='category', categoryorder='total descending')
fig1.update_traces(marker_color='rgb(255,165,0)', marker_line_color='rgb(255,140,0)',
                   marker_line_width=1.5, opacity=0.6, 
                   width=calculate_bar_width(len(df_company_sales)))
fig1.show()

# 例子2: 公司用户查询去年的销售情况
company = "湖北国贸能源化工有限公司"
last_year = current_year - 1
df_last_year = df[(df['销售结算日期'].dt.year == last_year) & (df['公司名称'] == company)]
df_monthly_sales = df_last_year.groupby(df_last_year['销售结算日期'].dt.to_period('M'))['销售金额'].sum().reset_index()
df_monthly_sales['销售结算日期'] = df_monthly_sales['销售结算日期'].dt.strftime('%Y-%m')

fig2 = px.bar(df_monthly_sales, x='销售结算日期', y='销售金额')
fig2 = set_mobile_layout(fig2, 
                         f'{company_short_names[company]}<br>{last_year}年月度销售情况', 
                         '月份', '销售额 (元)')
fig2.update_xaxes(type='category', categoryorder='category ascending')
fig2.update_traces(marker_color='rgb(255,165,0)', marker_line_color='rgb(255,140,0)',
                   marker_line_width=1.5, opacity=0.6, 
                   width=calculate_bar_width(len(df_monthly_sales)))
fig2.show()

# 例子3: 集团用户查询今年第一季度销售额大于3000万的公司
df_q1 = df[(df['销售结算日期'].dt.year == current_year) & (df['销售结算日期'].dt.quarter == 1)]
df_company_sales_q1 = df_q1.groupby('公司名称')['销售金额'].sum().reset_index()
df_company_sales_q1_filtered = df_company_sales_q1[df_company_sales_q1['销售金额'] > 30000000].sort_values('销售金额', ascending=False)
df_company_sales_q1_filtered['公司简称'] = df_company_sales_q1_filtered['公司名称'].map(company_short_names)

fig3 = px.bar(df_company_sales_q1_filtered, x='公司简称', y='销售金额')
fig3 = set_mobile_layout(fig3, 
                         f'{current_year}年第一季度<br>销售额大于3000万的公司', 
                         '公司名称', '销售额 (元)')
fig3.update_xaxes(type='category', categoryorder='total descending')
fig3.update_traces(marker_color='rgb(255,165,0)', marker_line_color='rgb(255,140,0)',
                   marker_line_width=1.5, opacity=0.6, 
                   width=calculate_bar_width(len(df_company_sales_q1_filtered)))
fig3.show()

# 例子4: 公司用户查询去年销售额大于300万的客户
df_last_year_company = df[(df['销售结算日期'].dt.year == last_year) & (df['公司名称'] == company)]
df_customer_sales = df_last_year_company.groupby('客户名称')['销售金额'].sum().reset_index()
df_customer_sales_filtered = df_customer_sales[df_customer_sales['销售金额'] > 3000000].sort_values('销售金额', ascending=False)

fig4 = px.bar(df_customer_sales_filtered, x='客户名称', y='销售金额')
fig4 = set_mobile_layout(fig4, 
                         f'{company_short_names[company]} {last_year}年<br>销售额大于300万的客户', 
                         '客户名称', '销售额 (元)')
fig4.update_xaxes(type='category', categoryorder='total descending')
fig4.update_traces(marker_color='rgb(255,165,0)', marker_line_color='rgb(255,140,0)',
                   marker_line_width=1.5, opacity=0.6, 
                   width=calculate_bar_width(len(df_customer_sales_filtered)))
fig4.show()

# 例子5: 公司用户查询今年销售额大于1000万的月份
df_this_year_company = df[(df['销售结算日期'].dt.year == current_year) & (df['公司名称'] == company)]
df_monthly_sales_this_year = df_this_year_company.groupby(df_this_year_company['销售结算日期'].dt.to_period('M'))['销售金额'].sum().reset_index()
df_monthly_sales_this_year['销售结算日期'] = df_monthly_sales_this_year['销售结算日期'].dt.strftime('%Y-%m')
df_monthly_sales_filtered = df_monthly_sales_this_year[df_monthly_sales_this_year['销售金额'] > 10000000]

fig5 = px.bar(df_monthly_sales_filtered, x='销售结算日期', y='销售金额')
fig5 = set_mobile_layout(fig5, 
                         f'{company_short_names[company]} {current_year}年<br>销售额大于1000万的月份', 
                         '月份', '销售额 (元)')
fig5.update_xaxes(type='category', categoryorder='category ascending')
fig5.update_traces(marker_color='rgb(255,165,0)', marker_line_color='rgb(255,140,0)',
                   marker_line_width=1.5, opacity=0.6, 
                   width=calculate_bar_width(len(df_monthly_sales_filtered)))
fig5.show()

In [12]:
# 将日期列转换为datetime类型
df['销售结算日期'] = pd.to_datetime(df['销售结算日期'])
df['采购结算日期'] = pd.to_datetime(df['采购结算日期'])

current_year = pd.Timestamp.now().year
last_year = current_year - 1

# 例子1-5保持不变

# 例子6: 集团用户查询各公司的采购额
df_q2 = df[(df['采购结算日期'].dt.year == current_year) & (df['采购结算日期'].dt.quarter == 2)]
df_company_purchase_q2 = df_q2.groupby('公司名称')['采购金额'].sum().sort_values(ascending=False).reset_index()
df_company_purchase_q2['公司简称'] = df_company_purchase_q2['公司名称'].map(company_short_names)

fig6 = px.bar(df_company_purchase_q2, x='公司简称', y='采购金额')
fig6 = set_mobile_layout(fig6, 
                         f'{current_year}年第二季度<br>各公司采购情况', 
                         '公司名称', '采购额 (元)')
fig6.update_xaxes(type='category', categoryorder='total descending')
fig6.update_traces(marker_color='rgb(255,165,0)', marker_line_color='rgb(255,140,0)',
                   marker_line_width=1.5, opacity=0.6, 
                   width=calculate_bar_width(len(df_company_purchase_q2)))
fig6.show()

# 例子7: 集团用户查询高毛利润公司
df_last_year = df[df['销售结算日期'].dt.year == last_year]
df_company_profit = df_last_year.groupby('公司名称')['毛利润'].sum().reset_index()
df_company_profit_filtered = df_company_profit[df_company_profit['毛利润'] > 10000000].sort_values('毛利润', ascending=False)
df_company_profit_filtered['公司简称'] = df_company_profit_filtered['公司名称'].map(company_short_names)

fig7 = px.bar(df_company_profit_filtered, x='公司简称', y='毛利润')
fig7 = set_mobile_layout(fig7, 
                         f'{last_year}年<br>毛利润超过1000万的公司', 
                         '公司名称', '毛利润 (元)')
fig7.update_xaxes(type='category', categoryorder='total descending')
fig7.update_traces(marker_color='rgb(255,165,0)', marker_line_color='rgb(255,140,0)',
                   marker_line_width=1.5, opacity=0.6, 
                   width=calculate_bar_width(len(df_company_profit_filtered)))
fig7.show()

# 例子8: 公司用户查询毛利率
company = "湖北国贸金属矿产有限公司"
df_this_year_company = df[(df['销售结算日期'].dt.year == current_year) & (df['公司名称'] == company)]
df_monthly_profit_rate = df_this_year_company.groupby(df_this_year_company['销售结算日期'].dt.to_period('M'))['毛利率'].mean().reset_index()
df_monthly_profit_rate['销售结算日期'] = df_monthly_profit_rate['销售结算日期'].dt.strftime('%Y-%m')
df_monthly_profit_rate['毛利率'] = df_monthly_profit_rate['毛利率'] * 100  # 转换为百分比

fig8 = px.line(df_monthly_profit_rate, x='销售结算日期', y='毛利率')
fig8 = set_mobile_layout(fig8, 
                         f'{company_short_names[company]}<br>{current_year}年月度毛利率', 
                         '月份', '毛利率 (%)')
fig8.update_xaxes(type='category', categoryorder='category ascending')
fig8.update_traces(line=dict(color='rgb(255,165,0)', width=2))
fig8.update_layout(yaxis_ticksuffix='%')  # 添加百分号到Y轴刻度
fig8.show()

# 例子9: 集团用户按条件查询采购额
df_half_year = df[(df['采购结算日期'].dt.year == current_year) & (df['采购结算日期'].dt.month <= 6)]
df_company_purchase = df_half_year.groupby('公司名称')['采购金额'].sum().reset_index()
df_company_purchase_filtered = df_company_purchase[df_company_purchase['采购金额'] > 70000000].sort_values('采购金额', ascending=False)  # 改为1000万
df_company_purchase_filtered['公司简称'] = df_company_purchase_filtered['公司名称'].map(company_short_names)

fig9 = px.bar(df_company_purchase_filtered, x='公司简称', y='采购金额')
fig9 = set_mobile_layout(fig9, 
                         f'{current_year}年上半年<br>采购额超过7000万元的公司', 
                         '公司名称', '采购额 (元)')
fig9.update_xaxes(type='category', categoryorder='total descending')
fig9.update_traces(marker_color='rgb(255,165,0)', marker_line_color='rgb(255,140,0)',
                   marker_line_width=1.5, opacity=0.6, 
                   width=calculate_bar_width(len(df_company_purchase_filtered)))
fig9.show()
# 例子10: 公司用户按商品类别查询销售额
company = "湖北国贸能源化工有限公司"
df_this_year_company_coal = df[(df['销售结算日期'].dt.year == current_year) & (df['公司名称'] == company) & (df['商品类别'] == '煤炭')]
df_monthly_sales_coal = df_this_year_company_coal.groupby(df_this_year_company_coal['销售结算日期'].dt.to_period('M'))['销售金额'].sum().reset_index()
df_monthly_sales_coal['销售结算日期'] = df_monthly_sales_coal['销售结算日期'].dt.strftime('%Y-%m')

fig10 = px.bar(df_monthly_sales_coal, x='销售结算日期', y='销售金额')
fig10 = set_mobile_layout(fig10, 
                          f'{company_short_names[company]}<br>{current_year}年煤炭月度销售额', 
                          '月份', '销售额 (元)')
fig10.update_xaxes(type='category', categoryorder='category ascending')
fig10.update_traces(marker_color='rgb(255,165,0)', marker_line_color='rgb(255,140,0)',
                    marker_line_width=1.5, opacity=0.6, 
                    width=calculate_bar_width(len(df_monthly_sales_coal)))
fig10.show()